## Initialize RDKit environment

In [0]:
! wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
! chmod +x Miniconda3-latest-Linux-x86_64.sh
! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
! conda install -q -y -c rdkit rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from keras.utils import np_utils
import matplotlib.pyplot as plt

from rdkit import Chem, rdBase
from rdkit.Chem import DataStructs, AllChem, RDConfig, rdMolDescriptors

np.set_printoptions(precision=3)

## Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Subroutines

In [0]:
def readMolecule( sdfname ) :
    moles = Chem.rdmolfiles.SDMolSupplier( sdfname )
    return moles

def removeInactiveMolecules( moles, actname ) :
    moles = [ m for m in moles if None != m.GetProp( actname ) and float( m.GetProp( actname ) ) > 0 ]
    return moles

def getFingerprintFromMolecule( moles, nBits=2048 ) :
    fps = [ rdMolDescriptors.GetMorganFingerprintAsBitVect( m, 2, nBits=nBits ) for m in moles ]
    np_fps = []
    for fp in fps:
        arr = np.zeros( ( 1, ) )
        #DataStructs.ConvertToNumpyArray( fp, arr )
        DataStructs.cDataStructs.ConvertToNumpyArray( fp, arr )
        np_fps.append( arr )
    return np.array( np_fps )



def getActivityOfMolecule( moles, actname ) :
    activity = [ m.GetProp( actname ) for m in moles ]
    activity = np.asarray( activity ).astype( 'float' )
    return activity


## DNN

In [0]:
def make_model( nfeatures ):
    model = Sequential()
    model.add( Dense( nfeatures, input_dim=nfeatures, activation='relu' ) )
    model.add( Dense( nfeatures, activation='relu' ) )
    model.add( Dense(1) )
    model.compile( loss='mean_squared_error', optimizer='adam', metrics=['mae' ] )
    return model

In [0]:
def plot_scatter( Y_train, Y_train_pred, Y_validation, Y_valid_pred ) :
    plt.figure(figsize=(6,6))
    plt.scatter(Y_train, Y_train_pred, color='black', s=2, label='training' )
    plt.scatter(Y_validation, Y_valid_pred, color='red', s=3, label='validation' )
    plt.xlabel( 'Experimental pIC50', labelpad=10 )
    plt.ylabel( 'Predicted pIC50', labelpad=10 )
    plt.xticks( np.arange( 2, 13 ) )
    plt.yticks( np.arange( 2, 13 ) )
    plt.savefig( 'qsar-scatter.png' )

In [0]:
def plot_history( hist ):
    fig, loss_ax = plt.subplots( figsize=(6, 4 ) )
    # fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')

    acc_ax.plot(hist.history['mae'], 'b', label='train MAE')
    acc_ax.plot(hist.history['val_mae'], 'g', label='val MAE')
    acc_ax.set_ylabel('MAE')
    acc_ax.legend(loc='upper right')

    plt.savefig( 'qsar-metric.png' )

In [0]:
def do_regression( X_train, Y_train, X_validation, Y_validation, epochs=50, batches=32, graph=False ):
    nfeatures = X_train.shape[1]
    model = make_model( nfeatures )
    # hist = model.fit( X_train, Y_train, epochs=iter, batch_size=bsize, validation_data=(X_validation, Y_validation), verbose=0 )
    hist = model.fit( X_train, Y_train, epochs=epochs, batch_size=batches, validation_split=0.2, verbose=0, callbacks=[  ] )

    loss_and_metric = model.evaluate( X_validation, Y_validation, verbose=0 )
    print( 'loss and metrics : %.3f %.3f' % (loss_and_metric[0], loss_and_metric[1]), ' : ', "%.3f" % hist.history['loss'][-1], ', ', "%.3f" % hist.history['val_loss'][-1] )

    if( graph == True ) :
        #predictions = model.predict( X_validation )
        #print( 'prediction shape : ', prediction.shape )

        # print( hist.history.keys() )
        plot_history( hist )

        Y_train_pred = model.predict(X_train).flatten()
        Y_valid_pred = model.predict(X_validation).flatten()
        plot_scatter( Y_train, Y_train_pred, Y_validation, Y_valid_pred )

    return loss_and_metric

In [0]:
def main() :
    sdfname = "./EGFR_8411.sdf"
    actname = 'PCHEMBL_VALUE'

    sdfname = "./sample_data/200303-pparg-chembl4qsar_6101.sdf"
    sdfname = "./drive/My Drive/ColabNotebooks/200303-pparg-chembl4qsar_6101.sdf"
    actname = "pChEMBL_Value"
    
    nBits = 2048
    frac_test = 0.3
    epochs = 30
    batsize = 10

    moles = readMolecule( sdfname )
    moles = removeInactiveMolecules( moles, actname )

    y = getActivityOfMolecule( moles, actname )
    x = getFingerprintFromMolecule( moles, nBits )

    maxiter=10
    results = []
    for trac_test in np.arange( 0.1, 0.9, 0.05 ) :
        for i in np.arange(0, maxiter) :
            x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=frac_test )
            # do_regression( x_train, y_train, x_test, y_test, epochs, batsize )
            loss_and_metric = do_regression( x_train, y_train, x_test, y_test, epochs )
            results.append( loss_and_metric )

    # print( "1:result=", results )
    # results = np.array(results)
    # print( "2:result=", results )

    plt.figure( figsize=( 8, 6 ) )
    plt.ylim( 0, 1 )
    plt.plot( results, label=( [ 'loss', 'mae' ] ) )
    plt.show()

if __name__ == '__main__' :
    main()
